In [ ]:
import pandas as pd
import numpy as np 
import seaborn as sns; sns.set_style('whitegrid')
from matplotlib import pyplot as plt
from scipy.stats import ttest_ind


In [ ]:
#import 2020 QB stats CSV
DF2020 = pd.read_csv('supp/2020QB.csv')
#splitting on '\' to keep player name
Name = DF2020['Player'].str.split('\\').str[0]
Name = Name.values.tolist()

#function to remove puncutaion from player names
def remove_punc(string):
    punc = '*+'
    for ele in string:
        if ele in punc:
            string = string.replace(ele,'')
    return string
#needed to add .strip() to lagging white space which affected merge downstream
Name = [remove_punc(i).strip() for i in Name]
DF2020['Player Name'] = Name
DF2020['Pos'] = DF2020['Pos'].str.upper()
DF2020 = DF2020.iloc[:, 2:]
DF2020['Year'] = '2020'



In [ ]:
#import 2019 QB stats CSV
DF2019 = pd.read_csv('supp/2019QB.csv')
#splitting on '\' to keep player name
Name = DF2019['Player'].str.split('\\').str[0]
Name = Name.values.tolist()

#function to remove puncutaion from player names
def remove_punc(string):
    punc = '*+'
    for ele in string:
        if ele in punc:
            string = string.replace(ele,'')
    return string
#needed to add .strip() to lagging white space which affected merge downstream
Name = [remove_punc(i).strip() for i in Name]
DF2019['Player Name'] = Name
DF2019['Pos'] = DF2019['Pos'].str.upper()
DF2019 = DF2019.iloc[:, 2:]
DF2019['Year'] = '2019'



In [ ]:
#import 2018 QB stats CSV
DF2018 = pd.read_csv('supp/2018QB.csv')
#splitting on '\' to keep player name
Name = DF2018['Player'].str.split('\\').str[0]
Name = Name.values.tolist()

#function to remove puncutaion from player names
def remove_punc(string):
    punc = '*+'
    for ele in string:
        if ele in punc:
            string = string.replace(ele,'')
    return string
#needed to add .strip() to lagging white space which affected merge downstream
Name = [remove_punc(i).strip() for i in Name]
DF2018['Player Name'] = Name
DF2018['Pos'] = DF2018['Pos'].str.upper()
DF2018 = DF2018.iloc[:, 2:]
DF2018['Year'] = '2018'

In [ ]:
#concat DF here with fields I want, all 3 seasons in 1 DF
DF_QB = pd.concat([DF2020,DF2019,DF2018])
DF_QB = DF_QB[['Player Name', 'Age', 'Pos', 'Tm', 'Year', 'G', 'GS', 'Cmp', 'Att', 'Cmp%' ,'Yds', 'TD']]
DF_QB.head(5)


In [ ]:
#Loading Player Measureables data
DFPM = pd.read_csv('supp/playermeasureables - Sheet1.csv')
Last_NamePM = DFPM['Player'].str.split(', ').str[0]
Last_NamePM = Last_NamePM.values.tolist()
First_NamePM = DFPM['Player'].str.split(', ').str[1]
First_NamePM = First_NamePM.values.tolist()
DFPM['First Name'] = First_NamePM
DFPM['Last Name'] = Last_NamePM
DFPM['Player Name'] = DFPM[['First Name', 'Last Name']].apply(lambda x: ' '.join(x), axis=1)

#Convert Height to Inches
def parse_ht(ht):
    # format: 7'0"
    ht_ = ht.split("'")
    ft_ = float(ht_[0])
    in_ = float(ht_[1].replace("\"",""))
    return int((12*ft_) + in_)

Height = DFPM["Height"].apply(lambda x:parse_ht(x))
DFPM['Height(IN)'] = Height
DFPM_Info = DFPM[['Player Name', 'Pos', 'Team', 'Height(IN)', 'Weight', 'DOB']]
DFPM_Info.head(5)



In [ ]:
#Merged 3 Season DF with player measureables DF
DF_QB_PM = pd.merge(DF_QB, DFPM_Info, how='left', on=['Player Name', 'Pos'])
#Filtered QB position and those that have greater than 60 passing attempts on the season
DF_QB_PM = DF_QB_PM[DF_QB_PM['Att'] > 60]
DF_QB_PM = DF_QB_PM[DF_QB_PM['Pos'] == 'QB']
DF_QB_PM = DF_QB_PM[DF_QB_PM['Height(IN)'].notnull()]


In [ ]:
Means_By_Year = DF_QB_PM.groupby('Year').mean()
Means_By_Year

In [ ]:
# #2018 data summary stats
DF_QB_PM_2018 = DF_QB_PM[DF_QB_PM['Year'] == '2018']
DF_QB_PM_2018.describe().transpose()




In [ ]:
#2019 data summary stats
DF_QB_PM_2019 = DF_QB_PM[DF_QB_PM['Year'] == '2019']
DF_QB_PM_2019.describe().transpose()


In [ ]:
#2020 data summary stats
DF_QB_PM_2020 = DF_QB_PM[DF_QB_PM['Year'] == '2020']
DF_QB_PM_2020.describe().transpose()
#DF_QB_PM_2018.head(5)

In [ ]:
#Distribution of QB heights for 2020
DF_QB_PM_2020['Height(IN)'].value_counts().sort_index().plot(kind = 'bar')
plt.title('Distribution of Quarterback Heights in NFL for 2020')
plt.xlabel('Height(IN)')
plt.ylabel('Number of Quarterbacks')

In [ ]:
#Distribution of QB heights for 2019
DF_QB_PM_2019['Height(IN)'].value_counts().sort_index().plot(kind = 'bar')
plt.title('Distribution of Quarterback Heights in NFL for 2019')
plt.xlabel('Height(IN)')
plt.ylabel('Number of Quarterbacks')

In [ ]:
#Distribution of QB heights for 2018
DF_QB_PM_2018['Height(IN)'].value_counts().sort_index().plot(kind = 'bar')
plt.title('Distribution of Quarterback Heights in NFL for 2018')
plt.xlabel('Height(IN)')
plt.ylabel('Number of Quarterbacks')

In [ ]:
#2020 line of best fit height to completions
sns.regplot(x=DF_QB_PM_2020['Height(IN)'], y=DF_QB_PM_2020['Cmp']);

In [ ]:
#2019 line of best fit  height to completions
sns.regplot(x=DF_QB_PM_2019['Height(IN)'], y=DF_QB_PM_2019['Cmp']);

In [ ]:
#2018 line of best fit height to completions
sns.regplot(x=DF_QB_PM_2018['Height(IN)'], y=DF_QB_PM_2018['Cmp']);

In [ ]:
#2020 line of best fit  height to attempts
sns.regplot(x=DF_QB_PM_2020['Height(IN)'], y=DF_QB_PM_2020['Att']);

In [ ]:
#2019 line of best fit  height to attempts
sns.regplot(x=DF_QB_PM_2019['Height(IN)'], y=DF_QB_PM_2019['Att']);

In [ ]:
#2019 line of best fit height to attempts
sns.regplot(x=DF_QB_PM_2018['Height(IN)'], y=DF_QB_PM_2018['Att']);

In [ ]:
#2020 line of best fit  height to completion percentage
sns.regplot(x=DF_QB_PM_2020['Height(IN)'], y=DF_QB_PM_2020['Cmp%']);

In [ ]:
#2019 line of best fit  height to completion percentage
sns.regplot(x=DF_QB_PM_2019['Height(IN)'], y=DF_QB_PM_2019['Cmp%']);

In [ ]:
#2018 line of best fit  height to completion percentage
sns.regplot(x=DF_QB_PM_2018['Height(IN)'], y=DF_QB_PM_2018['Cmp%']);

In [ ]:
#short quarterbacks: 73 inches and shorter 
DF_Short = DF_QB_PM[DF_QB_PM['Height(IN)']<74]
DF_Short_Test = DF_Short[['TD']]
#tall quarterbacks: 64 inches and taller
DF_Tall = DF_QB_PM[DF_QB_PM['Height(IN)']>73]
DF_Tall_Test = DF_Tall[['TD']]

# T-test 
ttest_ind(DF_Short_Test,DF_Tall_Test)




In [ ]:
# Returned: Ttest_indResult(statistic=array([0.28508418]), pvalue=array([0.7761015])) 